In [ ]:
class BookFormat(Enum):
  HARDCOVER, PAPERBACK, AUDIO_BOOK, EBOOK, NEWSPAPER, MAGAZINE, JOURNAL = 1,2,3,4,5,6,7

class BookStatus(Enum):
  AVAILABLE, RESERVED, LOANED, LOST = 1, 2, 3, 4

class ReservationStatus(Enum):
  WAITING, PENDIND, CANCELLED, COMPLETED = 1, 2, 3, 4

class AccountStatus(Enum):
  ACTIVE, CLOSED, CANCELLED, BLACKLISTED, NONE = 1, 2, 3, 4, 5

In [ ]:
from abc import ABC

class Address:
  def __init__(self, street, city, state, zip_code, country):
    pass

class Person(ABC):
  def __init__(self, name, address, email, phone):
    pass

class Constants:
  self.MAX_BOOKS_PER_USER = 5
  self.MAX_LENDING_DAYS = 10
  

In [ ]:
from abc import ABC, abstractmethod

# Account is a super class for both Librarian and Member
class Account(ABC):
  def __init__(self, id, password, person, status=AccountStatus.ACTIVE): # default active
    pass

  def reset_password(self):
    pass

class Librarian(Account):
  def __init(self, id, password, person, status=AccountStatus.ACTIVE):
    super().__init__(id, password, person, status)
  
  def add_book_item(self, book_item): #
    pass 

  def block_member(self, member):
    pass

  def unblock_member(self, member):
    pass

class Member(Account):
  def __init(self, id, password, person, status=AccountStatus.ACTIVE):
    super().__init__(id, password, person, status)
    self.__date_of_membership = datetime.date.today()
    self.__total_books_checked_out = 0
  
  def get_total_books_checked_out():
    return self.__total_books_checked_out

  def reserve_book_item(self, book_item):
    pass

  def increment_total_books_checked_out(self):
    self.__total_books_checked_out += 1

  def renew_book_item(self, book_item):
    pass

  # Logic for checking out a book item
  def checkout_book_item(self, book_item):
    # First, check current Member's total books
    if self.get_total_books_checked_out() > Constants.MAX_BOOKS_PER_USER:
      print(f"User limited to {Constants.MAX_BOOKS_PER_USER} books only!")
      return False
    
    # Next, check book item's reservation status using its barcode
    book_reservation_status = BookReservation.fetch_reservation_details(book_item.get_barcode())

    # If book item has been reserved by another member
    if book_reservation_status and book_reservation_status.get_member_id() != self.id:
      print("Book reserved by another memeber")
      return False
    # The book is reserved by this member
    elif book_reservation_status:
      book_reservation_status.update_status(ReservationStatus.COMPLETED)
    
    # Checkout and mark this book as LOANED
    if not book_item.checkout(self.get_id()):
      return False

    self.increment_total_books_checked_out()
    return True
  
  # check if there's should be any fine associated with this book item when user return
  def check_for_fine(self, book_item_barcode):
    # First, get book lending record
    book_lending = BookLending.fetch_lending_details(book_item_barcode)
    due_date = book_lending.get_due_date()
    today = datetime.date.today()

    if due_date > today:
      diff_in_days = (today - due_date).days
      Fine.collect_fine(self.get_id(), diff_in_days)

  # Logic for returning book item
  def return_book_item(self, book_item):
    # Fine user if returned late
    self.check_for_fine(book_item.get_barcode()) 

    # Check for any reservation of the current book item
    book_reservation = BookReservation.fetch_reservation_details(book_item.get_barcode())

    if book_reservation:
      # Update status to RESERVED and send out available notification
      book_item.update_book_item_status(BookStatus.RESERVED)
      book_reservation.send_book_available_notification()
    else:
      book_item.update_book_item_status(BookStatus.AVAILABLE)
  
  # Logic to renew this book item
  def renew_book_item(self, book_item):
    # First, fine user if date exceeds due date
    self.check_for_fine(book_item.get_barcode())

    # Next, check reservation status to see whether renewal is accepted
    book_reservation = BookReservation.fetch_reservation_details(
        book_item.get_barcode())

    if book_reservation and book_reservation.get_member_id() != self.get_id():
      print("Book is reserved by another member!")
      self.decrement_total_books_checked_out()
      book_item.update_book_item_status(BookStatus.RESERVED)
      book_reservation.send_book_available_notification()
      return False
    elif book_reservation: # Book item is being reserved by this member
      book_reservation.update_status(ReservationStatus.RESERVED)
    
    BookLending.lend_book(book_item.get_barcode(), self.get_id()) # Mark lending to this user
    book_item.update_due_date(
      datetime.datetime.now().AddDays(Constants.MAX_LENDING_DAYS)
    )

    return True